In [3]:
import pandas as pd

# TEST NOTEBOOK TO IMPLEMENT THE TKAN MODEL
"""
from model.TKAN import Model
a = Model()
a.compile_execute_test()
"""

'\nfrom model.TKAN import Model\na = Model()\na.compile_execute_test()\n'

In [1]:
"""
import os
import pandas as pd
from datetime import datetime

# Define the directory containing the Meteocat files
meteocat_dir = 'data/processed/meteocat/'

# Get the current month and year
current_month = datetime.now().month
current_year = datetime.now().year

# Loop through all files in the Meteocat directory
for filename in os.listdir(meteocat_dir):
    if 'daily_all' in filename and filename.endswith('.csv'):
        file_path = os.path.join(meteocat_dir, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Convert the 'data' column to datetime
        df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')

        # Filter out the rows for the current month and year
        df_filtered = df[~((df['data'].dt.month == current_month) & (df['data'].dt.year == current_year))]

        # Save the filtered DataFrame back to the CSV file
        df_filtered.to_csv(file_path, index=False)
"""